In [1]:
from src.data.dataset_functions import make_dataframe_from_rawcsv,provide_rawcsv

provide_rawcsv(True)
new_dataframe = make_dataframe_from_rawcsv()

providing ../data/raw/2022_place_canvas_history-000000000003.csv ...
not found. need to download ../data/raw/2022_place_canvas_history-000000000003.csv.gzip ...
downloading from https://placedata.reddit.com/data/canvas-history/2022_place_canvas_history-000000000003.csv.gzip to ../data/raw/2022_place_canvas_history-000000000003.csv.gzip
unpacking ../data/raw/2022_place_canvas_history-000000000003.csv.gzip into ../data/raw/2022_place_canvas_history-000000000003.csv
deleting ../data/raw/2022_place_canvas_history-000000000003.csv.gzip
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/vscode/.ivy2/cache
The jars for the packages stored in: /home/vscode/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-daafe7fd-09c9-494a-814f-881f5348236b;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 78ms :: artifacts dl 4ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------------------

In [2]:
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from src.data.dataset_functions import transform_dataframe_colums,transform_dataframe_normalize_seconds,transform_dataframe_timestamp

def transform_dataframe(df_input: DataFrame) -> DataFrame:
    ''' Transforms a dataframe from its source structure into something more usable '''
    df_transform = transform_dataframe_filter_nomods(df_input)
    df_transform = transform_dataframe_timestamp(df_transform)
    df_transform = transform_dataframe_normalize_seconds(df_transform)
    df_transform = transform_dataframe_colums(df_transform)
    return df_transform

def transform_dataframe_filter_nomods(df_input: DataFrame) -> DataFrame:
    ''' filters mod data - they have 2 coordinate pairs per row '''
    df_output = df_input.where(F.size(F.split('coordinate', ',')) == 2)
    return df_output

def transform_dataframe_filter_onlymods(df_input: DataFrame) -> DataFrame:
    ''' outputs only mod data - 2 coordinate pairs '''
    df_output = df_input.where(F.size(F.split('coordinate', ',')) > 2)
    return df_output

def transform_dataframe_modcolumns(df_input: DataFrame) -> DataFrame:
    ''' Transforms columns from [\'timestamp\',\'user_id\',\'pixel_color\',\'coordinate\']
    into [\'user_id\',\'x1\',\'y1\',\'x2\',\'y2\',\'t\',\'pixel_color\']'''
    df_output = df_input.select('user_id',
                                F.split('coordinate', ',').getItem(
                                    0).cast('int').alias('x1'),
                                F.split('coordinate', ',').getItem(
                                    1).cast('int').alias('y1'),
                                F.split('coordinate', ',').getItem(
                                    2).cast('int').alias('x2'),
                                F.split('coordinate', ',').getItem(
                                    3).cast('int').alias('y2'),
                                F.col('timestamp').alias('t'),
                                'pixel_color')
    return df_output

def transform_dataframe_onlymods(df_input: DataFrame) -> DataFrame:
    ''' Transfroms a dataframe from its source structure but uses only the rows with moderator data '''
    df_transform = transform_dataframe_filter_onlymods(df_input)
    df_transform = transform_dataframe_timestamp(df_transform)
    df_transform = transform_dataframe_normalize_seconds(df_transform)
    df_transform = transform_dataframe_modcolumns(df_transform)
    return df_transform


In [3]:

transformed_dataframe = transform_dataframe(new_dataframe)

In [4]:
mod_dataframe = transform_dataframe_onlymods(new_dataframe)

In [5]:
mod_dataframe.show()

+--------------------+----+----+----+----+----------+-----------+
|             user_id|  x1|  y1|  x2|  y2|         t|pixel_color|
+--------------------+----+----+----+----+----------+-----------+
|q+XjkQ6WRx0aBLtb2...|1349|1718|1424|1752|1649035370|    #000000|
|gS0DWvPgaiQkHvG4N...| 297|1750| 364|1813|1649028592|    #FFB470|
|gXOuxCyb/dslyRb5G...|  23|1523| 172|1792|1649045540|    #000000|
|Tqn87vyPxAHtPMu4x...|  51|1691| 154|1807|1649046581|    #000000|
|oPv1iZrj4UMjfhzOT...|  44|1652| 165|1899|1649045770|    #000000|
|m8NEcPbf5XRV5ppeu...| 298|1805| 329|1839|1649027104|    #FFB470|
|LKS2u3QL2N3Olv7rn...| 257|1736| 296|1780|1649027330|    #FFB470|
|q/Dk6lmcXm8bcDbNI...| 298|1770| 334|1803|1649027009|    #FFB470|
|HkR0yRQUJ1wsjh4Zo...| 251|1805| 296|1812|1649027436|    #FFF8B8|
|7JiQyrONpFJphvBEP...| 271|1835| 296|1859|1649027571|    #FFF8B8|
+--------------------+----+----+----+----+----------+-----------+



In [6]:
mods = mod_dataframe.drop("x1").drop("y1").drop("x2").drop("y2").drop("t").drop("pixel_color")
mods = mods.collect()
print(mods)


[Row(user_id='q+XjkQ6WRx0aBLtb2xRGWBrsHALXejJxEE2hs6sDJHfN2LO0X0ML5Yy2CFsNdFxU6CgXTrrSKv/Un6Dx6JdHbA=='), Row(user_id='gS0DWvPgaiQkHvG4NsHveLvpn8uf50t+sOY3nIykDvkyEdsDvqiTB5Fzh4XNe+26OKIhkmogXOaSzoZ6IXS+2Q=='), Row(user_id='gXOuxCyb/dslyRb5GQW+JbG9NRxMHxGGV3V5018ha4bVrUYUCw2LDwbsvicgI8bkWmlwPg1Y68SQK8aEJ30a6w=='), Row(user_id='Tqn87vyPxAHtPMu4xuAqTHkBvUhFi/BKLnQ0XyEraQ3ut93qO9zY3nUivplx4NVxKrvTS3xDGVV2+GicCuV0kQ=='), Row(user_id='oPv1iZrj4UMjfhzOTVNZKwNe6u/bNG2Tabw8H7UOABps62BJuj+/TA5zA7C47pqMEOCxlloedL4+EQgRkiP48w=='), Row(user_id='m8NEcPbf5XRV5ppeuZ3KLIYAG8GuHkNIxOEsCD06Ey8I1E5QicEqMzXkHrZQeVg6lIagpzwsaq7k0/Y/o5h5PA=='), Row(user_id='LKS2u3QL2N3Olv7rnUCWry4KJ5K4Ea+/9qKyadTNl01apEcCr8iYdnmiamfJE7OjkLQHZ69YYiMpqg4bBpCnCA=='), Row(user_id='q/Dk6lmcXm8bcDbNIhDglz7kFuCmX6zkca9UPivDix5WWi/ILMHHn0Ljw+Nwzh/DcRoub9ti6LBEAp9fpJek4Q=='), Row(user_id='HkR0yRQUJ1wsjh4Zo4VdKE43IctIGMFS9VuVm9IyCFcPOAPS4mTuhwb4UFs4ifeuhmknaCQlBG+f44AITi2VQA=='), Row(user_id='7JiQyrONpFJphvBEPVUGyxjBsdvU8fuiSVzpMuTkx